In [2]:
import os
import mysql.connector as MariaDB


## Building the Database
Setting up the Mysql.Connector object to serve as an access to connect to the database.


In [3]:
# establish connection to MariaDB
db_user = os.environ.get('DB_USER')
db_password = os.environ.get('DB_PASSWORD')
# getting db user and db pass from PATH variables

In [5]:
sql_connection = MariaDB.connect(
    user=db_user,
    password=db_password,
    host='localhost'
)

Initializing the cursor object, which communicates with the SQL server.

In [6]:
cursor = sql_connection.cursor()

Creating DB


In [7]:
try: 
    cursor.execute("""
    CREATE DATABASE db_capstone DEFAULT CHARACTER SET 'utf8'

    """)
except MariaDB.Error as error:
    print(f"Failed to Create Database, {error}")



Creating Tables with dictionary

In [15]:
# maria db docs https://mariadb.com/kb/en/documentation/

db_capstone_tables = {}

db_capstone_tables['CDW_SAPP_CUSTOMER'] = """
    CREATE TABLE CDW_SAPP_CUSTOMER (
    cust_id INT PRIMARY KEY AUTO_INCREMENT,
    ssn VARCHAR(9) NOT NULL UNIQUE,
    first_name VARCHAR(255) NOT NULL,
    middle_name VARCHAR(255),
    last_name VARCHAR(255) NOT NULL,
    credit_card_no VARCHAR(16) NOT NULL,
    full_street_address VARCHAR(225) NOT NULL,
    cust_city VARCHAR(25) NOT NULL,
    cust_state VARCHAR(2) NOT NULL,
    cust_country VARCHAR(225) NOT NULL,
    cust_zip VARCHAR(5) NOT NULL,
    cust_phone VARCHAR(13),
    cust_email VARCHAR(255),
    last_updated TIMESTAMP
    )
"""
# SAPP_CUSTOMER

# ssid not null and unique
# opted to use ssn as varchar considering we are not
# using ssn for computation

# also opted to generate a primary key instead of using 
# ssn as a primary key

# notes on creating tables
# note the CREATE TABLE [tablename] (column, type, ~constraint) syntax

db_capstone_tables['CDW_SAPP_BRANCH'] = """
    CREATE TABLE CDW_SAPP_BRANCH (
    branch_code INT PRIMARY KEY AUTO_INCREMENT,
    branch_name VARCHAR(225) NOT NULL,
    branch_street VARCHAR(225) NOT NULL,
    branch_city VARCHAR(225) NOT NULL,
    branch_state VARCHAR(2) NOT NULL,
    branch_zip VARCHAR(5) NOT NULL,
    branch_phone VARCHAR(13) NOT NULL,
    last_updated TIMESTAMP
    )

"""
# SAPP_BRANCH

# added primary key


db_capstone_tables['CDW_SAPP_CREDIT_CARD'] = """
    CREATE TABLE CDW_SAPP_CREDIT_CARD (
    transaction_id INT PRIMARY KEY AUTO_INCREMENT,
    timeid DATE NOT NULL,
    credit_card_no VARCHAR(16) NOT NULL,
    cust_ssn VARCHAR(9) NOT NULL,
    branch_code INT NOT NULL,
    transaction_type VARCHAR(225) NOT NULL,
    transaction_value DOUBLE NOT NULL
    )
"""

# SAPP_CREDIT_CARD

# transaction_value
# debated making transaction values unsigned b/c negative values
# are w.r.t what side of the transaction you're on.
# decided to leave them in for now. 

# timeid
# opted to make timeid TIME instead of varchar, simply makes more sense
# easier to perform queries with


Iterating through table dictionary

In [16]:
cursor.execute("USE db_capstone")

for table in db_capstone_tables:
    try: 
        cursor.execute(db_capstone_tables[table])
    except MariaDB.Error as error:
        print(f"Error while creating {table}:\n{error}")

# will iterate through all k:v pairs
# will continue to attempt CREATE TABLE statement even if
# one or more fail


With the code above, I've now created a SQL Database with the schema as shown below:

<div>
    <img src='https://github.com/The-Captain-William/Per-Scholas-Capstone/blob/main/images/capstone_physical_model.png?raw=true'>
</div>

Where 
- 'ssn' from sapp_customer relates to 'cust_ssn' from sapp_credit_card
- 'branch_code' from sapp_branch realtes to 'branc_code' from sapp_credit_card

📝 Note: 
<p>I debated if I should use the 'ssn' from sapp_customer as the primary key, and rename 'cust_ssn' to 'ssn' so joining tables can be slightly easier, 
<br>but I decided it would be more appropriate to stick with what we have here.</p>